In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/ncullen93/torchsample
!pip install tensorboardX 
#!pip install --upgrade torch
!pip install torch==1.5.1+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!wget https://gist.githubusercontent.com/archie9211/ae3c8411da88ae8b2a05b0ee1a4fd412/raw/ee1a6e78fc498ad6d4830cd2eb6033839235ea8a/colab-ssh-jupyter.sh
!pip install -U git+https://github.com/pytorch/vision

  Cloning https://github.com/ncullen93/torchsample to /tmp/pip-req-build-w2xyznnm
  Running command git clone -q https://github.com/ncullen93/torchsample /tmp/pip-req-build-w2xyznnm
  Created wheel for torchsample: filename=torchsample-0.1.3-cp36-none-any.whl size=43417 sha256=5bbcddac37a8f80e25ba3787dc4dd707471f2cd31c7885251f816454006e2599
  Stored in directory: /tmp/pip-ephem-wheel-cache-f8imob58/wheels/88/c7/72/14cd9a173eed1e29d0b17d866e7d9ee511d31a834aedd27489
Successfully built torchsample
     |████████████████████████████████| 204kB 6.8MB/s 
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 26kB/s 
ERROR: torchvision 0.6.0+cu101 has requirement torch==1.5.0, but you'll have torch 1.5.1+cu101 which is incompatible.
  Found existing installation: torch 1.5.0+cu101
    Uninstalling torch-1.5.0+cu101:
      Successfully uninstalled torch-1.5.0+cu101
--2020-06-21 13:29:35--  https://gist.githubusercontent.com/archie92

In [ ]:
import pdb
import torch
import torch.nn as nn
from torchvision import models
import numpy as np
import os
import pickle
import torch.nn.functional as F
import torch.utils.data as data
from torchsample.transforms import RandomRotate, RandomTranslate, RandomFlip, ToTensor, Compose, RandomAffine
from torchvision import transforms
import torchvision.transforms.functional as TF
from torch.autograd import Variable
import argparse
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import json
from datetime import datetime
from pathlib import Path
from sklearn import metrics

In [ ]:
class MRNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.alexnet(pretrained=True)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(256, 1)

    def forward(self, x):
        x = torch.squeeze(x, dim=0) # only batch size 1 supported
        x = self.model.features(x)
        x = self.gap(x).view(x.size(0), -1)
        x = torch.max(x, 0, keepdim=True)[0]
        x = self.classifier(x)
        return x


In [ ]:
class MRI_alex(nn.Module):
  def __init__(self, training=True):
    super().__init__()
    self.axial_net = models.alexnet(pretrained=training)
    self.sagit_net = models.alexnet(pretrained=training)
    self.coron_net = models.alexnet(pretrained=training)

    self.gap_axial = nn.AdaptiveAvgPool2d(1)
    self.gap_sagit = nn.AdaptiveAvgPool2d(1) 
    self.gap_coron = nn.AdaptiveAvgPool2d(1)
    self.classifier = nn.Linear(3*256, 1)

  def forward(self,vol_axial, vol_sagit, vol_coron):
    vol_axial = torch.squeeze(vol_axial, dim=0)
    vol_sagit = torch.squeeze(vol_sagit, dim=0)
    vol_coron = torch.squeeze(vol_coron, dim=0)

    vol_axial = self.axial_net.features(vol_axial)
    vol_sagit = self.sagit_net.features(vol_sagit)
    vol_coron = self.coron_net.features(vol_coron)

    vol_axial = self.gap_axial(vol_axial).view(vol_axial.size(0), -1)
    x = torch.max(vol_axial, 0, keepdim=True)[0]

    vol_sagit = self.gap_sagit(vol_sagit).view(vol_sagit.size(0), -1)
    y = torch.max(vol_sagit, 0, keepdim=True)[0]

    vol_coron = self.gap_coron(vol_coron).view(vol_coron.size(0), -1)
    z = torch.max(vol_coron, 0, keepdim=True)[0]

    w = torch.cat((x, y, z), 1)
    out = self.classifier(w)
    return out

In [ ]:
train_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomHorizontalFlip(),
            transforms.RandomAffine(25, translate=(0.1, 0.1)),
            transforms.ToTensor()
        ])

In [ ]:
valid_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor()
        ])

In [ ]:
def preprocess(series):
   series = torch.tensor(np.stack((series,)*3, axis=1))
   series = np.dot(np.divide((series - series.min()) , (series.max() - series.min())) , MAX_PIXEL_VAL)
   series = np.divide((series - MEAN) , STDDEV)
   return series

In [ ]:
INPUT_DIM = 224
MAX_PIXEL_VAL = 255
MEAN = 58.09
STDDEV = 49.73
class Dataset(data.Dataset):
    def __init__(self, datadir, tear_type, use_gpu , transform=None):
        super().__init__()
        self.use_gpu = use_gpu
        label_dict = {}
        self.paths = []
        self.transform = transform
        
        abnormal_label_dict = {}
        if datadir[-1]=="/":
            datadir = datadir[:-1]
        self.datadir = datadir
        for i, line in enumerate(open(datadir+'-'+tear_type+'.csv').readlines()):
            line = line.strip().split(',')
            filename = line[0]
            label = line[1]
            label_dict[filename] = int(label)
        

        for i, line in enumerate(open(datadir+'-'+"abnormal"+'.csv').readlines()):
            line = line.strip().split(',')
            filename = line[0]
            label = line[1]
            abnormal_label_dict[filename] = int(label)

        for filename in os.listdir(os.path.join(datadir, "axial")):
            if filename.endswith(".npy"):
                self.paths.append(filename)
        
        self.labels = [label_dict[path.split(".")[0]] for path in self.paths]
        self.abnormal_labels = [abnormal_label_dict[path.split(".")[0]] for path in self.paths]

        #neg_weight = np.mean(self.labels)
        #self.weights = [neg_weight, 1 - neg_weight]

        if tear_type != "abnormal":
            temp_labels = [self.labels[i] for i in range(len(self.labels)) if self.abnormal_labels[i]==1]
            neg_weight = np.mean(temp_labels)
        else:
            neg_weight = np.mean(self.labels)
        
        self.weights = [neg_weight, 1 - neg_weight]
    

    def weighted_loss(self, prediction, target):
        weights_npy = np.array([self.weights[int(t[0])] for t in target.data])
        weights_tensor = torch.FloatTensor(weights_npy)
        if self.use_gpu:
            weights_tensor = weights_tensor.cuda()
        loss = F.binary_cross_entropy_with_logits(prediction, target, weight=Variable(weights_tensor))
        return loss

    def __getitem__(self, index):
        filename = self.paths[index]
        vol_axial = np.load(os.path.join(self.datadir, "axial", filename))
        vol_sagit = np.load(os.path.join(self.datadir, "sagittal", filename))
        vol_coron = np.load(os.path.join(self.datadir, "coronal", filename))


        # axial
        pad = int((vol_axial.shape[2] - INPUT_DIM)/2)
        vol_axial = vol_axial[:,pad:-pad,pad:-pad]
        #vol_axial = np.dot(np.divide((vol_axial - vol_axial.min()) , (vol_axial.max() - vol_axial.min())) , MAX_PIXEL_VAL)
        #vol_axial = np.divide((vol_axial - MEAN) , STDDEV)
        #vol_axial = np.stack((vol_axial,)*3, axis=1)
        vol_axial = preprocess(vol_axial)
        
        vol_axial_tensor = torch.FloatTensor(vol_axial)
          
        
        # sagittal
        pad = int((vol_sagit.shape[2] - INPUT_DIM)/2)
        vol_sagit = vol_sagit[:,pad:-pad,pad:-pad]
        #vol_sagit = np.dot(np.divide((vol_sagit - vol_sagit.min()) , (vol_sagit.max() - vol_sagit.min())) , MAX_PIXEL_VAL)
        #vol_sagit = np.divide((vol_sagit - MEAN) , STDDEV)
        #vol_sagit = np.stack((vol_sagit,)*3, axis=1)
        vol_sagit = preprocess(vol_sagit)
        
        
        vol_sagit_tensor = torch.FloatTensor(vol_sagit)
        
        # coronal
        pad = int((vol_coron.shape[2] - INPUT_DIM)/2)
        vol_coron = vol_coron[:,pad:-pad,pad:-pad]
        #vol_coron = np.dot(np.divide((vol_coron - vol_coron.min()) , (vol_coron.max() - vol_coron.min())) , MAX_PIXEL_VAL)
        #vol_coron = np.divide((vol_coron - MEAN) , STDDEV)
        #vol_coron = np.stack((vol_coron,)*3, axis=1)
        vol_coron = preprocess(vol_coron)
        
        vol_coron_tensor = torch.FloatTensor(vol_coron)
        

        label_tensor = torch.FloatTensor([self.labels[index]])

        return vol_axial_tensor, vol_sagit_tensor, vol_coron_tensor, label_tensor , self.abnormal_labels[index]

    def __len__(self):
        return len(self.paths)


In [ ]:
import pandas as pd 
def compare():
    data = pd.read_csv('/content/drive/My Drive/MRNet-v1.0/train-acl.csv' , header=None) 

    for i in range(len(train_dataset)):
      if train_dataset.labels[i] != data[1][int(train_dataset.paths[i][:4])]:
        print("Not Equal")
        break
      else:
        print(i , train_dataset.labels[i] , data[1][int(train_dataset.paths[i][:4])] ,train_dataset.abnormal_labels[i] ,  train_dataset.paths[i] )



In [ ]:
def load_data(task, use_gpu):
  train_dir = "/content/drive/My Drive/MRNet-v1.0/train"
  valid_dir = "/content/drive/My Drive/MRNet-v1.0/valid"

  train_dataset = Dataset(train_dir, task, use_gpu  )
  valid_dataset = Dataset(valid_dir, task, use_gpu )

  train_loader = data.DataLoader(train_dataset, batch_size=1, num_workers=11, shuffle=True)
  valid_loader = data.DataLoader(valid_dataset, batch_size=1, num_workers=11, shuffle=False)

  return train_loader, valid_loader

In [ ]:
def run_model(model, loader, train=False, optimizer=None,abnormal_model_path=None):
    preds = []
    labels = []

    if train:
        model.train()
    else:
        if abnormal_model_path:
            abnormal_model = MRI_alex()
            state_dict = torch.load(abnormal_model_path)
            abnormal_model.load_state_dict(state_dict)
            abnormal_model.cuda()
            abnormal_model.eval()
        model.eval()

    total_loss = 0.0
    num_batches = 0

    for batch in tqdm(loader):
        vol_axial, vol_sagit, vol_coron, label, abnormal = batch
        if train:
            if abnormal_model_path and not abnormal:
                continue
            optimizer.zero_grad()
        if loader.dataset.use_gpu:
            vol_axial, vol_sagit, vol_coron = vol_axial.cuda(), vol_sagit.cuda(), vol_coron.cuda()
            label = label.cuda()
        vol_axial, vol_sagit, vol_coron = Variable(vol_axial), Variable(vol_sagit), Variable(vol_coron)
        label = Variable(label)

        logit = model.forward(vol_axial, vol_sagit, vol_coron)

        loss = loader.dataset.weighted_loss(logit, label)
        total_loss += loss.item()

        pred = torch.sigmoid(logit)

        pred_npy = pred.data.cpu().numpy()[0][0]

        if abnormal_model_path and not train:
            abnormal_logit = abnormal_model.forward(vol_axial,vol_sagit,vol_coron)
            abnormal_pred = torch.sigmoid(abnormal_logit)
            abnormal_pred_npy = abnormal_pred.data.cpu().numpy()[0][0]
            pred_npy = pred_npy * abnormal_pred_npy

        label_npy = label.data.cpu().numpy()[0][0]

        preds.append(pred_npy)
        labels.append(label_npy)

        if train:
            loss.backward()
            optimizer.step()
        num_batches += 1

    avg_loss = total_loss / num_batches
    
    fpr, tpr, threshold = metrics.roc_curve(labels, preds)
    auc = metrics.auc(fpr, tpr)

    if abnormal_model_path and not train:
        del abnormal_model

    return avg_loss, auc, preds, labels


In [ ]:
def evaluate(split, model_path, diagnosis, use_gpu):
    train_loader, valid_loader, test_loader = load_data(diagnosis, use_gpu)

    model = MRNet()
    state_dict = torch.load(model_path, map_location=(None if use_gpu else 'cpu'))
    model.load_state_dict(state_dict)

    if use_gpu:
        model = model.cuda()

    if split == 'train':
        loader = train_loader
    elif split == 'valid':
        loader = valid_loader
    elif split == 'test':
        loader = test_loader
    else:
        raise ValueError("split must be 'train', 'valid', or 'test'")

    loss, auc, preds, labels = run_model(model, loader)

    print(f'{split} loss: {loss:0.4f}')
    print(f'{split} AUC: {auc:0.4f}')

    return preds, labels

In [ ]:
def train(rundir, task, epochs, learning_rate, use_gpu,model,abnormal_model_path=None):
    train_loader, valid_loader = load_data(task, use_gpu)
    
    model = model
    '''for dirpath, dirnames, files in os.walk(rundir):
        if not files:
            break
        max_epoch = 0
        model_path = None
        for fname in files:
            if fname.endswith(".json"):
                continue
            ep = int(fname[27:])
            if ep >= max_epoch:
                max_epoch = ep
                model_path = os.path.join(dirpath, fname)
        
        if model_path:
            state_dict = torch.load(model_path, map_location=(None if use_gpu else 'cpu'))
            model.load_state_dict(state_dict)
'''
    if use_gpu:
        model = model.cuda()

    optimizer = torch.optim.Adam(model.parameters(), learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.3, threshold=1e-4)

    best_val_loss = float('inf')

    start_time = datetime.now()

    for epoch in range(epochs):
        change = datetime.now() - start_time
        print('starting epoch {}. time passed: {}  : lr = {} '.format(epoch+1, str(change) , learning_rate))
        
        train_loss, train_auc, _, _ = run_model(model, train_loader, train=True, optimizer=optimizer , abnormal_model_path=abnormal_model_path)

        print(f'train loss: {train_loss:0.4f}')
        print(f'train AUC: {train_auc:0.4f}')

        val_loss, val_auc, _, _ = run_model(model, valid_loader , abnormal_model_path=abnormal_model_path)
        
        print(f'valid loss: {val_loss:0.4f}')
        print(f'valid AUC: {val_auc:0.4f}')

        scheduler.step(val_loss)

In [ ]:
model_abnormal=MRI_alex(training=True)
model_ab = train('/content/drive/My Drive/MRNet-v1.0',
                       task='abnormal',
                       epochs=50,
                       learning_rate=1e-06,
                       use_gpu=True,
                       model=model_abnormal)

In [ ]:
model_ACL=MRI_alex(training=True)
model_meniscus = train('/content/drive/My Drive/MRNet-v1.0',
                       task='ACL',
                       epochs=50,
                       learning_rate=1e-06,
                       use_gpu=True,
                       model=model_ACL, 
                       abnormal_model_path='/content/drive/My Drive/MRNet-v1.0/Untitled Folder/abnormal.pt')

In [ ]:
model_men=MRI_alex(training=True)
model_meniscus = train('/content/drive/My Drive/MRNet-v1.0',
                       task='meniscus',
                       epochs=50,
                       learning_rate=1e-06,
                       use_gpu=True,
                       model=model_men, 
                       abnormal_model_path='/content/drive/My Drive/MRNet-v1.0/Untitled Folder/abnormal.pt')

In [ ]:
torch.save(model_abnormal.state_dict(),'/content/drive/My Drive/MRNet-v1.0/Untitled Folder/abnormal.pt')

In [ ]:
torch.save(model_ACL.state_dict() ,'/content/drive/My Drive/MRNet-v1.0/Untitled Folder/ACL.pt' )

In [ ]:
torch.save(model_men.state_dict() ,'/content/drive/My Drive/MRNet-v1.0/Untitled Folder/meniscus.pt' )